In [1]:
# load the dataset and take a quick view
import pandas as pd

df = pd.read_csv("enterpret.csv")
df.head()

,ID,URL,Source,Type,CreatedAt,Language,Record Sentiment,Tracked Keywords,Reasons,Content,...,metadata.Total_Feedback_Volume_Call_Recording__c,metadata.Metadata_Summary,metadata.Metadata_CallParticipants_name,metadata.Metadata_PublicURL,metadata.Metadata_Tags_[],metadata.Metadata_CallParticipants_email,metadata.Metadata_Owners_[],metadata.Metadata_EndTime,metadata.Metadata_StartTime,metadata.Influenced_By__c
0,e0e5cedb-2da1-5476-bfef-366eebc71656,https://dashboard.enterpret.com/enterpretinc/r...,Slack,RecordTypeConversation,2024-01-12T22:28:45Z,eng,NEGATIVE,NaN,Issue With Decreased Feedback Volume From Sources,Agent: Mike McNasby jil It looks like the volu...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,a879cfc1-120c-5a69-b059-5820f08abae3,https://dashboard.enterpret.com/enterpretinc/r...,Gong,RecordTypeAudioRecording,2024-01-12T21:39:45Z,eng,NaN,"Feedback, Integration, Quantify, Reporting, Sl...","Ability To Filter Information In Zendesk, Help...",<AUDIO_CONTENT>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,69949ac0-6280-5a97-be88-315985fe2cbc,https://dashboard.enterpret.com/enterpretinc/r...,Slack,RecordTypeConversation,2024-01-12T20:57:40Z,eng,NEUTRAL,Schedule,NaN,"User: Adding Daniela González, Emma's ExecOps ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7396e171-235d-5cd4-af82-7b8d4c70ce76,https://dashboard.enterpret.com/enterpretinc/r...,Salesforce - Opportunity,RecordTypeSurvey,2024-01-12T20:57:28Z,eng,NEGATIVE,Feedback,"Issue With Low Volume, Issue With Feedback Goi...","Closed Lost Notes\nNot enough volume, not enou...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6794583a-f498-5904-925e-08b826581709,https://dashboard.enterpret.com/enterpretinc/r...,Gong,RecordTypeAudioRecording,2024-01-12T20:01:16Z,eng,NaN,"Product Design, Feedback, Enterpret, Survey, D...","Ability To Get More Detailed Feedback, Ability...",<AUDIO_CONTENT>,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# check the columns
print(f"Columns: {df.columns}")
print(f"Rows and Columns: {df.shape}")

Columns: Index(['ID', 'URL', 'Source', 'Type', 'CreatedAt', 'Language',
       'Record Sentiment', 'Tracked Keywords', 'Reasons', 'Content',
       ...
       'metadata.Total_Feedback_Volume_Call_Recording__c',
       'metadata.Metadata_Summary', 'metadata.Metadata_CallParticipants_name',
       'metadata.Metadata_PublicURL', 'metadata.Metadata_Tags_[]',
       'metadata.Metadata_CallParticipants_email',
       'metadata.Metadata_Owners_[]', 'metadata.Metadata_EndTime',
       'metadata.Metadata_StartTime', 'metadata.Influenced_By__c'],
      dtype='object', length=104)
Rows and Columns: (904, 104)


In [3]:
# check columns which are all empty so that can be dropped
empty_cols = [col for col in df.columns if df[col].isnull().all()]
print(f"Empty columns: {empty_cols} \nNo. of empty columns: {len(empty_cols)} ")

Empty columns: ['metadata.Post_Title', 'metadata.Author_Name', 'metadata.Subreddit', 'metadata.Post_Author', 'metadata.Post_URL', 'metadata.Author_Type', 'metadata.CreatedAt', 'metadata.Post_ID', 'metadata.Author_Flair', 'metadata.Tags_[]', 'metadata.Author_Tags_[]', 'metadata.Post_Flair'] 
No. of empty columns: 12 


In [4]:
# drop columns having all null rows
df.dropna(axis=1, how="all", inplace=True)

In [5]:
# now checking the shape again
df.shape

(904, 92)

In [6]:
# Checking the unique values of columns that appear to have categorical features
unique_sources = df["Source"].unique()
unique_types = df["Type"].unique()
unique_languages = df["Language"].unique()
unique_sentiments = df["Record Sentiment"].unique()

print("Unique sources:", unique_sources, len(unique_sources))
print("Unique types:", unique_types, len(unique_types))
print("Unique languages:", unique_languages, len(unique_languages))
print("Unique sentiments:", unique_sentiments, len(unique_sentiments))

Unique sources: ['Slack' 'Gong' 'Salesforce - Opportunity' 'Grain'] 4
Unique types: ['RecordTypeConversation' 'RecordTypeAudioRecording' 'RecordTypeSurvey'] 3
Unique languages: ['eng'] 1
Unique sentiments: ['NEGATIVE' nan 'NEUTRAL' 'POSITIVE'] 4


In [7]:
source_count = df["Source"].value_counts().head(4)
type_count = df["Type"].value_counts().head(3)
sentiment_count = df["Record Sentiment"].value_counts().head(3)

In [8]:
# getting the distribution
print(source_count)
print()
print(type_count)
print()
print(sentiment_count)

Source
Gong                        506
Slack                       341
Salesforce - Opportunity     56
Grain                         1
Name: count, dtype: int64

Type
RecordTypeAudioRecording    507
RecordTypeConversation      341
RecordTypeSurvey             56
Name: count, dtype: int64

Record Sentiment
NEGATIVE    108
NEUTRAL      76
POSITIVE      9
Name: count, dtype: int64


In [9]:
null_content_count = df["Reasons"].isnull().sum()
print(f"Number of null rows in Reasons column: {null_content_count}")

Number of null rows in Reasons column: 403


In [10]:
# analysing the protagonist further
audio_content_count = df[df['Content'] == '<AUDIO_CONTENT>'].shape[0]
null_content_count = df['Content'].isnull().sum()
print(f"Number of columns where Content column is <AUDIO_CONTENT>: {audio_content_count}")
print(f"Number of columns where Content column is Null: {null_content_count}")


Number of columns where Content column is <AUDIO_CONTENT>: 507
Number of columns where Content column is Null: 44


In [11]:
df_summary_not_empty = df[df['Summary'].notnull()]
summary_not_empty_count = df_summary_not_empty.shape[0]
df_reasons_not_empty = df[df['Reasons'].notnull()]
reasons_not_empty_count = df_reasons_not_empty.shape[0]
print(f"Number of rows where Reasons column is not empty: {reasons_not_empty_count}")
print(f"Number of rows where Summary column is not empty: {summary_not_empty_count}")

Number of rows where Reasons column is not empty: 501
Number of rows where Summary column is not empty: 217


In [12]:
# Check the 'Type' of rows where 'Reasons' column is not empty
df_reasons_not_empty_types = df_reasons_not_empty['Type'].value_counts()
print(f"Types of rows where Reasons column is not empty: \n{df_reasons_not_empty_types}")

Types of rows where Reasons column is not empty: 
Type
RecordTypeAudioRecording    443
RecordTypeConversation       56
RecordTypeSurvey              2
Name: count, dtype: int64


In [13]:
# Check how many rows of type 'RecordTypeAudioRecording' have their 'Reasons' column empty
df_audio_recording_empty_reasons = df[(df['Type'] == 'RecordTypeAudioRecording') & (df['Reasons'].isnull())]
audio_recording_empty_reasons_count = df_audio_recording_empty_reasons.shape[0]
print(f"Number of rows of type 'RecordTypeAudioRecording' with empty 'Reasons' column: {audio_recording_empty_reasons_count}")

Number of rows of type 'RecordTypeAudioRecording' with empty 'Reasons' column: 64


## Create a new dataframe

* For potentially finetuning a model on the data
* For providing input to the Claim Extractor

In [14]:
# as per the problem statement, select desired columns
main_df = df[
    ["ID", "URL", "Type", "Source", "CreatedAt", "Summary", "Reasons", "Content"]
]

In [15]:
main_df.to_csv('data.csv')

In [ ]:
from nltk.tokenize import sent_tokenize
import re
import json
def split_content_based_on_type(row):
    content = row["Content"]
    type = row["Type"]

    # Check if content is a string, not <AUDIO_CONTENT>, and not None
    if type == "RecordTypeAudioRecording":
            return "<AUDIO_CONTENT>"
    
    elif isinstance(content, str) and content is not None:    
        if type == "RecordTypeConversation":
            # Split the content based on 'User:' or 'Agent:'
            dialogues = re.split(r"(?=User:|Agent:)", content)
            # Remove leading/trailing whitespaces from each dialogue
            dialogues = [dialogue.strip() for dialogue in dialogues if dialogue != '']
            # Tokenize sentences within each dialogue and assign each sentence a key
            sentence_dict = {}
            sentence_index = 1
            for dialogue in dialogues:
                sentences = sent_tokenize(dialogue)
                for sentence in sentences:
                    sentence_dict[sentence_index] = sentence
                    sentence_index += 1
            sentence_dict = json.dumps(sentence_dict)
            return sentence_dict
        elif type == "RecordTypeSurvey":
            # Split the content into sentences
            sentences = sent_tokenize(content)
            sentences = {index+1: sentence for index, sentence in enumerate(sentences)}
            sentences = json.dumps(sentences)
            return sentences
    else:
        return ''

In [ ]:
main_df["Content"] = main_df.apply(split_content_based_on_type, axis=1)

In [ ]:
for i, content in enumerate(main_df['Content']):
        print(f"Content {i+1}: NumDialogues: - {content}\n")
        break

In [ ]:
import json

def csv_to_json(jsonl_file_path):

    df = main_df
    # Convert each row of the DataFrame to JSON and write to a file
    with open(jsonl_file_path, "w") as f:
        for _, row in df.iterrows():
            json.dump(row.to_dict(), f)
            f.write("\n")


csv_to_json("./enterpret.jsonl")

In [ ]:
# Save the DataFrame to a CSV file
main_df.to_csv("main_df.csv", index=False)

In [ ]:
# Easier to work with JSON
class Content:
    def __init__(self, dict):
        self.id = dict["ID"]
        self.url = dict["URL"]
        self.type = dict["Type"]
        self.timestamp = dict["CreatedAt"]
        self.source = dict["Source"]
        self.content = dict["Content"]
        self.summary = dict["Summary"]
        self.reason = dict["Reasons"]

    @property
    def is_conversation(self):
        return self.type == "RecordTypeConversation"
    
    @property
    def is_survey(self):
        return self.type == "RecordTypeSurvey"